In [ ]:
import os
import osmnx as ox
import networkx as nx
import pickle
import openrouteservice
import folium
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import numpy as np
import heapq
from math import radians, cos, sin, sqrt, atan2

# Initialize OpenRouteService client
API_KEY = "5b3ce3597851110001cf6248d909b0549b614bde8aac0f381c393ba2"
client = openrouteservice.Client(key=API_KEY)

# ----------------------------------------
# 1. Geocoding Function
# ----------------------------------------
def get_coordinates(place_name):
    result = client.pelias_search(place_name)
    if result and "features" in result and result["features"]:
        coords = result["features"][0]["geometry"]["coordinates"]
        return coords[1], coords[0]  # (lat, lon)
    return None

# Example delivery locations (Replace with actual locations from your dataset)
locations = [
    "Rajpur Road, Dehradun",
    "ISBT, Dehradun",
    "Paltan Bazar, Dehradun",
    "Graphic Era Deemed to be University, Dehradun",
    "Prem Nagar, Dehradun",
    "Graphic Era Hill University, Dehradun"
]

# Convert place names to coordinates
coords_array = np.array([get_coordinates(loc) for loc in locations])  # Convert locations to coordinates

# Ensure k < number of locations
K_range = range(2, min(len(coords_array), 7))  # Adjusted range

inertia = []
silhouette_scores = []

for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    cluster_labels = kmeans.fit_predict(coords_array)
    
    inertia.append(kmeans.inertia_)  # Inertia
    if k < len(coords_array):  # Avoid silhouette error
        silhouette_scores.append(silhouette_score(coords_array, cluster_labels))
    else:
        silhouette_scores.append(None)  # Placeholder for invalid k

# Find the best k (highest silhouette score)
optimal_k = K_range[np.argmax(silhouette_scores)]  # Selecting k with the highest Silhouette Score
print(f"Optimal number of clusters (k): {optimal_k}")
import pandas as pd

# Assuming you already have cluster_labels from KMeans
# Combine original data into a DataFrame
clustered_data = pd.DataFrame({
    'Location': locations,
    'Latitude': coords_array[:, 0],
    'Longitude': coords_array[:, 1],
    'Cluster': cluster_labels
})

# Save to CSV
clustered_data.to_csv('clustered_locations.csv', index=False)

print("✅ Clustered data saved to 'clustered_locations.csv'")



In [ ]:
import pandas as pd

# Load all clustered data (if already saved)
clustered_data = pd.read_csv("clustered_locations.csv")

# Select one cluster (e.g., cluster 0)
cluster_id = 0  # You can loop through or choose based on optimization needs
cluster_points = clustered_data[clustered_data['Cluster'] == cluster_id]

# Convert to list of (lat, lon) tuples
cluster_coords = list(zip(cluster_points['Latitude'], cluster_points['Longitude']))



In [ ]:
import os
import pickle
import pandas as pd
import numpy as np
import osmnx as ox
import networkx as nx
import folium
from math import radians, cos, sin, sqrt, atan2

# ------------------------------
# CONFIGURATION
# ------------------------------
GRAPH_FILE = "dehradun.graphml"
GEOCACHE_FILE = "geocache.pkl"

# ------------------------------
# 1. Load Graph from File or OSM
# ------------------------------
def load_graph():
    if os.path.exists(GRAPH_FILE):
        print("✅ Loading graph from file...")
        return ox.load_graphml(GRAPH_FILE)
    else:
        print("⏳ Downloading Dehradun map...")
        G = ox.graph_from_place("Dehradun, India", network_type='drive')
        ox.save_graphml(G, GRAPH_FILE)
        return G

# ------------------------------
# 2. Haversine Distance (meters)
# ------------------------------
def haversine(coord1, coord2):
    R = 6371000
    lat1, lon1 = map(radians, coord1)
    lat2, lon2 = map(radians, coord2)
    dlat = lat2 - lat1
    dlon = lon2 - lon1

    a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c

# ------------------------------
# 3. Haversine Heuristic for A*
# ------------------------------
def haversine_heuristic(u, v, G):
    lat1, lon1 = G.nodes[u]['y'], G.nodes[u]['x']
    lat2, lon2 = G.nodes[v]['y'], G.nodes[v]['x']
    R = 6371
    dlat = radians(lat2 - lat1)
    dlon = radians(lat2 - lon1)
    a = sin(dlat / 2)**2 + cos(radians(lat1)) * cos(radians(lat2)) * sin(dlon / 2)**2
    c = 2 * atan2(sqrt(a), sqrt(1 - a))
    return R * c * 1000

# ------------------------------
# 4. Load CSV and Extract Coordinates
# ------------------------------
def load_locations(csv_file):
    df = pd.read_csv(csv_file)
    coords = df[['Latitude', 'Longitude']].values
    return df['Location'].tolist(), coords

# ------------------------------
# 5. TSP Nearest Neighbor Heuristic
# ------------------------------
def tsp_nearest_neighbor(locations, coords):
    centroid = np.mean(coords, axis=0)
    start_index = np.argmin([haversine(centroid, c) for c in coords])

    visited = [start_index]
    current_index = start_index

    while len(visited) < len(coords):
        nearest = None
        min_dist = float('inf')
        for i in range(len(coords)):
            if i not in visited:
                dist = haversine(coords[current_index], coords[i])
                if dist < min_dist:
                    nearest = i
                    min_dist = dist
        visited.append(nearest)
        current_index = nearest

    return visited

# ------------------------------
# 6. Plot A* Paths Between TSP Nodes
# ------------------------------
def plot_astar_route_from_coords(coords, order):
    G = load_graph()
    m = folium.Map(location=coords[order[0]], zoom_start=13)

    for idx in range(len(order) - 1):
        start_coord = coords[order[idx]]
        end_coord = coords[order[idx + 1]]

        start_node = ox.nearest_nodes(G, X=start_coord[1], Y=start_coord[0])
        end_node = ox.nearest_nodes(G, X=end_coord[1], Y=end_coord[0])

        astar_path = nx.astar_path(G, start_node, end_node,
                                   heuristic=lambda u, v: haversine_heuristic(u, v, G),
                                   weight='length')

        path_coords = [(G.nodes[n]['y'], G.nodes[n]['x']) for n in astar_path]
        folium.PolyLine(path_coords, color='green', weight=4, opacity=0.7,
                        tooltip=f"A* Segment {idx+1}").add_to(m)

    # Add markers
    for idx, i in enumerate(order):
        folium.Marker(
            location=coords[i],
            popup=f"{idx+1}.",
            icon=folium.Icon(color='blue' if idx > 0 else 'red')
        ).add_to(m)

    return m

# ------------------------------
# 7. Main Execution Function
# ------------------------------
def run_tsp_with_map(csv_file):
    locations, coords = load_locations(csv_file)
    order = tsp_nearest_neighbor(locations, coords)

    print("📍 TSP Visit Order:")
    for i in order:
        print(f"➡ {locations[i]}")

    # Save ordered route
    ordered_df = pd.DataFrame({
        'VisitOrder': list(range(1, len(order)+1)),
        'Location': [locations[i] for i in order],
        'Latitude': [coords[i][0] for i in order],
        'Longitude': [coords[i][1] for i in order]
    })
    ordered_df.to_csv("tsp_route.csv", index=False)
    print("\n✅ Route saved to 'tsp_route.csv'")

    # Show A* map using OSM
    return plot_astar_route_from_coords(coords, order)

# ------------------------------
# 8. Run It
# ------------------------------
if __name__ == "__main__":
    map_object = run_tsp_with_map("clustered_locations.csv")
    map_object.save("tsp_astar_map.html")
    print("🗺️ Map saved as 'tsp_astar_map.html'")
